In [1]:
import torch
torch.cuda.empty_cache()

In [3]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

n_eposides_tr=5  #训练集迭代生成多少个任务
n_eposides_va=10
img_size=30  #处理后的数据集图片大小
max_epoch=10
save_epoch=2
shot=1
query=15
train_way=30
test_way=5
save_path='./save_protonet'

# 数据集处理

In [35]:
import os.path as osp
from PIL import Image

from torch.utils.data import Dataset
from torchvision import transforms


ROOT_PATH = 'D:\\数据库\\小样本数据库\\mini-imagenet\\mini-imagenet'


class MiniImageNet(Dataset):

    def __init__(self, setname):
        csv_path = osp.join(ROOT_PATH, setname + '.csv')
        #filename                 label
        #n0153282900000005.jpg    n01532829
        lines = [x.strip() for x in open(csv_path, 'r').readlines()][1:]

        data = []  #图片
        label = []  #图片对应的类别索引
        lb = -1

        self.wnids = []  #记录该数据集都有哪些类，每个元素（类名）的索引对应label

        for l in lines:
            name, wnid = l.split(',')
            path = osp.join(ROOT_PATH, 'images', name)  #每张图片的地址
            if wnid not in self.wnids:
                self.wnids.append(wnid)
                lb += 1
            data.append(path)
            label.append(lb)

        self.data = data
        self.label = label
        
        #print(len(self.data))  #训练集一共38400个样本，测试集12000，验证集9600
        #print(len(self.wnids)) #训练集包含64个类别，测试集20类，验证集16类，每类有600个样本
        #print(self.data[0])  #self.data包含每个样本图片的地址  D:\数据库\小样本数据库\mini-imagenet\mini-imagenet\images\n0153282900000005.jpg
        #print(self.label[0])  #self.label包含每个样本的类别 [0~63]


        self.transform = transforms.Compose([  #图片转换
            transforms.Resize(img_size),
            transforms.CenterCrop(img_size),  #84
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        path, label = self.data[i], self.label[i]
        image = self.transform(Image.open(path).convert('RGB'))
        return image, label

In [36]:
import torch
import numpy as np


class CategoriesSampler():

    def __init__(self, label, n_batch, n_cls, n_per):
        self.n_batch = n_batch
        self.n_cls = n_cls  #每个任务包含的样本类别数
        self.n_per = n_per  #每个任务每个类别包含的样本数=s+q

        label = np.array(label)  #数据集的列别标签
        self.m_ind = []  #按类别储存所有样本的索引
        for i in range(max(label) + 1):  #遍历数据集中的所有类
            ind = np.argwhere(label == i).reshape(-1)  #找出数据集中某个类别的样本索引
            ind = torch.from_numpy(ind)
            self.m_ind.append(ind)   #把每一个样本的索引按类别存放一起，后组成一个数组
        #print(self.m_ind)  #[[类别一的600个样本的索引]，[类别二,,,,,,,[]]   

    def __len__(self):
        return self.n_batch  #有多少个batch
    
    def __iter__(self):  
        for i_batch in range(self.n_batch):  #每次next(iter())生成一个eposide数据，1个eposide包含n_cls*（s+q）个样本,一个loader一共可以产生n_batch个eposide
            batch = []
            classes = torch.randperm(len(self.m_ind))[:self.n_cls]  #从数据集中随机选择n_cls个类
            for c in classes:
                l = self.m_ind[c]  #被选出的某个类的所有样本索引
                pos = torch.randperm(len(l))[:self.n_per]  #从选出的每个类中随机取出n_per个样本
                batch.append(l[pos])  
            batch = torch.stack(batch).t().reshape(-1)  #batch保存一个eposide所需的所有样本的索引，注意.t()是按（s+q）个存放
            yield batch  #返回一个n way k shot任务需要的样本，即n*(s+q)

In [37]:
#训练集
trainset = MiniImageNet('train')
#每次生成100个batch=100个任务？每个batch包含支持集和查询集
train_sampler = CategoriesSampler(trainset.label, n_eposides_tr,train_way, shot + query)
train_loader = DataLoader(dataset=trainset, batch_sampler=train_sampler,num_workers=0, pin_memory=True)
#每次迭代分别产生480个（样本vs标签），480=30*(15+1), iter(train_loader))[0]的shape为[480, 3, 84, 84])
#print(len(next(iter(train_loader))[0]))  #480
#print(len(iter(train_loader)))  #100

#验证集
valset = MiniImageNet('val')
val_sampler = CategoriesSampler(valset.label, n_eposides_va,test_way, shot + query)
val_loader = DataLoader(dataset=valset, batch_sampler=val_sampler,num_workers=0, pin_memory=True)

In [40]:
# a=0
# for i, batch in enumerate(train_loader, 1):
#     a=i
#     data, label = [_.cuda() for _ in batch]
#     p = shot * train_way  #每个任务的支持集样本数
#     data_shot, data_query = data[:p], data[p:]  #一个任务的支持集样本和查询集样本
#     label_shot, label_query = label[:p], label[p:]
#     #print(label)  #[16, 19, 59, 63, 20,,随机30个数，重复15遍]，所以前p个样本为支持集，后面的样本为查询集
#     #print(len(data_shot))  #30
#     #print(len(data)) #480
#     #print(data_query[0].shape)  #([3, 28, 28])
#     #print(len(label)) #450=30*15
    
# print(a) #10=n_batch

2

# 工具函数

In [7]:
import os
import shutil
import time
import pprint

import torch


def set_gpu(x):
    os.environ['CUDA_VISIBLE_DEVICES'] = x
    print('using gpu:', x)


def ensure_path(path):
    if os.path.exists(path):
        if input('{} exists, remove? ([y]/n)'.format(path)) != 'n':
            shutil.rmtree(path)
            os.makedirs(path)
    else:
        os.makedirs(path)


class Averager():

    def __init__(self):
        self.n = 0
        self.v = 0

    def add(self, x):
        self.v = (self.v * self.n + x) / (self.n + 1)
        self.n += 1

    def item(self):
        return self.v


def count_acc(logits, label):
    pred = torch.argmax(logits, dim=1)
    return (pred == label).type(torch.cuda.FloatTensor).mean().item()


def dot_metric(a, b):
    return torch.mm(a, b.t())


def euclidean_metric(a, b):
    n = a.shape[0]
    m = b.shape[0]
    a = a.unsqueeze(1).expand(n, m, -1)
    b = b.unsqueeze(0).expand(n, m, -1)
    logits = -((a - b)**2).sum(dim=2)
    return logits


class Timer():

    def __init__(self):
        self.o = time.time()  #记录起始时间

    def measure(self, p=1): #返回训练模型花费的时间
        x = (time.time() - self.o) / p
        x = int(x)
        if x >= 3600:
            return '{:.1f}h'.format(x / 3600)
        if x >= 60:
            return '{}m'.format(round(x / 60))
        return '{}s'.format(x)

_utils_pp = pprint.PrettyPrinter()
def pprint(x):
    _utils_pp.pprint(x)


def l2_loss(pred, label):
    return ((pred - label)**2).sum() / len(pred) / 2



# 构造模型

In [8]:
import torch.nn as nn


def conv_block(in_channels, out_channels):
    bn = nn.BatchNorm2d(out_channels)
    nn.init.uniform_(bn.weight) # 随机初始化bn层的参数
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        bn,
        nn.ReLU(),
        nn.MaxPool2d(2)
    )


class Convnet(nn.Module):

    def __init__(self, x_dim=3, hid_dim=64, z_dim=64):
        super().__init__()
        self.encoder = nn.Sequential(  #84=》42=》21=》10=》5
            conv_block(x_dim, hid_dim),  
            conv_block(hid_dim, hid_dim),
            conv_block(hid_dim, hid_dim),
            conv_block(hid_dim, z_dim),  
        )
        self.out_channels = 1600  #1600=5*5*64  经过卷积运算后拉直的结果

    def forward(self, x):
        x = self.encoder(x)
        return x.view(x.size(0), -1)   #拉直卷积运算后的特征shape为（？,1600)


In [9]:
#实例化网络
model = Convnet().cuda()
    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  #优化器
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)  #学习率调整器

# 训练模型

In [10]:
#把训练后的模型参数保存为args.save_path, name + '.pth'
    def save_model(name):
        torch.save(model.state_dict(), osp.join(save_path, name + '.pth'))

In [11]:
#记录训练每个batch的结果
trlog = {}
trlog['train_loss'] = []
trlog['val_loss'] = []
trlog['train_acc'] = []
trlog['val_acc'] = []
trlog['max_acc'] = 0.0  #记录模型最佳准确数

timer = Timer()  #用于计算训练模型花费的时间
                        #200+1
for epoch in range(1, max_epoch+1):   #训练模型迭代200轮
    lr_scheduler.step()  #初始化学习率调整器

    #模型训练
    model.train()

    tl = Averager()  #计算训练时在每个任务损失和准确数的均值
    ta = Averager()

    for i, batch in enumerate(train_loader, 1):  #此循环会进行n_batch次，每次产生一个eposide所需的数据=(s+q)*cls
        data, _ = [_.cuda() for _ in batch]   #分别把每个任务的数据data都搬到gpu上，data表示一个任务的数据=(s+q)*cls
        p = shot * train_way  #每个任务的支持集样本数
        data_shot, data_query = data[:p], data[p:]  #一个任务的支持集样本和查询集样本

        proto = model(data_shot)  #输出（支持集样本数=shot*train_way，？*？*64）
        proto = proto.reshape(shot, train_way, -1).mean(dim=0)  #计算支持集每类样本的样本均值作为该类的原型（train_way，？*？*64）

        label = torch.arange(train_way).repeat(query)  #（train_way，q）  初始化查询集每个样本对应每个类别的分数
        label = label.type(torch.cuda.LongTensor)

        logits = euclidean_metric(model(data_query), proto)  #计算查询集每个样本到每个类别原型的欧式距离，（train_way*q）
        loss = F.cross_entropy(logits, label)  #计算交叉熵损失
        acc = count_acc(logits, label)  #计算预测准率
        print('epoch {}, train {}/{}, loss={:.4f} acc={:.4f}'.format(epoch, i, len(train_loader), loss.item(), acc))

        #把一个任务的损失和准确数加入到tl和ta，用于计算在当前所有任务上的损失和准确数的均值
        tl.add(loss.item())
        ta.add(acc)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
        proto = None; logits = None; loss = None

    #n_batch个episode上的平均损失和准确数
    tl = tl.item()
    ta = ta.item()

    #模型验证，每n_batch个任务（train_loader走完一遍）训练完后，验证一次模型
    model.eval()

    vl = Averager()
    va = Averager()
    
    for i, batch in enumerate(val_loader, 1):
        data, _ = [_.cuda() for _ in batch]
        p = shot * test_way
        data_shot, data_query = data[:p], data[p:]
        #with torch.no_grad():？？
        proto = model(data_shot)
        proto = proto.reshape(shot, test_way, -1).mean(dim=0)

        label = torch.arange(test_way).repeat(query)
        label = label.type(torch.cuda.LongTensor)

        logits = euclidean_metric(model(data_query), proto)
        loss = F.cross_entropy(logits, label)
        acc = count_acc(logits, label)

        vl.add(loss.item())
        va.add(acc)
            
        proto = None; logits = None; loss = None

    vl = vl.item()
    va = va.item()
    print('epoch {}, val, loss={:.4f} acc={:.4f}'.format(epoch, vl, va))

    # 记录模型最佳
    if va > trlog['max_acc']:
        trlog['max_acc'] = va
        save_model('max-acc')  #保存获得最佳准确率的模型

    trlog['train_loss'].append(tl)
    trlog['train_acc'].append(ta)
    trlog['val_loss'].append(vl)
    trlog['val_acc'].append(va)

    torch.save(trlog, osp.join(save_path, 'trlog'))

    save_model('epoch-last')  #保存最终的训练模型

    if epoch % save_epoch == 0:  #每save_epoch个epoch保存一次模型
        save_model('epoch-{}'.format(epoch))

    print('ETA:{}/{}'.format(timer.measure(), timer.measure(epoch / max_epoch)))

epoch 1, train 1/5, loss=9.9470 acc=0.0467
epoch 1, train 2/5, loss=6.7594 acc=0.0933
epoch 1, train 3/5, loss=5.8546 acc=0.0400
epoch 1, train 4/5, loss=4.9547 acc=0.0689
epoch 1, train 5/5, loss=4.1162 acc=0.0756
epoch 1, val, loss=1.6084 acc=0.2267
ETA:17s/3m
epoch 2, train 1/5, loss=4.2205 acc=0.0444
epoch 2, train 2/5, loss=3.6415 acc=0.0867
epoch 2, train 3/5, loss=3.7704 acc=0.0556
epoch 2, train 4/5, loss=3.5180 acc=0.0711
epoch 2, train 5/5, loss=3.4403 acc=0.0689
epoch 2, val, loss=1.5972 acc=0.2587
ETA:33s/3m
epoch 3, train 1/5, loss=3.4601 acc=0.0733
epoch 3, train 2/5, loss=3.4962 acc=0.0622
epoch 3, train 3/5, loss=3.4852 acc=0.0467
epoch 3, train 4/5, loss=3.4033 acc=0.0489
epoch 3, train 5/5, loss=3.2918 acc=0.0622
epoch 3, val, loss=1.5964 acc=0.2813
ETA:51s/3m
epoch 4, train 1/5, loss=3.2911 acc=0.0800
epoch 4, train 2/5, loss=3.4070 acc=0.0556
epoch 4, train 3/5, loss=3.3050 acc=0.0933
epoch 4, train 4/5, loss=3.3226 acc=0.0778
epoch 4, train 5/5, loss=3.3460 acc=0.0

# 测试模型

In [13]:
load='./save_protonet/max-acc.pth'
batch=5
way=5
shot=1
query=30

dataset = MiniImageNet('test')
sampler = CategoriesSampler(dataset.label,batch, way, shot + query)
loader = DataLoader(dataset, batch_sampler=sampler,num_workers=0, pin_memory=True)

#导出训练得最佳准确数的模型
model = Convnet().cuda()
model.load_state_dict(torch.load(load))
#模型验证
model.eval()

ave_acc = Averager()

for i, batch in enumerate(loader, 1):
    data, _ = [_.cuda() for _ in batch]
    k = way * shot
    data_shot, data_query = data[:k], data[k:]

    x = model(data_shot)  #在支持集上前传的结果
    x = x.reshape(shot, way, -1).mean(dim=0)  #计算支持集每类样本的均值==》原型
    p = x

    logits = euclidean_metric(model(data_query), p)

    label = torch.arange(way).repeat(query)
    label = label.type(torch.cuda.LongTensor)

    acc = count_acc(logits, label)
    ave_acc.add(acc)
    print('batch {}: {:.2f}({:.2f})'.format(i, ave_acc.item() * 100, acc * 100))
        
    x = None; p = None; logits = None

batch 1: 26.67(26.67)
batch 2: 31.33(36.00)
batch 3: 30.89(30.00)
batch 4: 29.83(26.67)
batch 5: 29.20(26.67)
